<a href="https://colab.research.google.com/github/p-tech/wbs-dm/blob/main/sqlite-exercise/SQLite_DB_DataSet_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1: CREATE the SQLite database;**


We need to import the sqlite3 module and create the database and tables.  You'll see this follows the syntax we have used on previous weeks.


In [1]:
import sqlite3

#This statement creates a connection labelled as conn.  This will be used throughout to ensure the consistency for when we start to query the database tables.
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE olist_customers (
    customer_id VARCHAR(32) PRIMARY KEY,
    customer_unique_id VARCHAR(32),
    customer_zip_code_prefix INT,
    customer_city VARCHAR(255),
    customer_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_geolocation (
    geolocation_zip_code_prefix INT,
    geolocation_lat FLOAT,
    geolocation_lng FLOAT,
    geolocation_city VARCHAR(255),
    geolocation_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE olist_order_items (
    order_id VARCHAR(32),
    order_item_id INT,
    product_id VARCHAR(32),
    seller_id VARCHAR(32),
    shipping_limit_date DATETIME,
    price FLOAT,
    freight_value FLOAT,
    PRIMARY KEY (order_id, order_item_id)
);
''')

cursor.execute('''
CREATE TABLE olist_order_payments (
    order_id VARCHAR(32),
    payment_sequential INT,
    payment_type VARCHAR(50),
    payment_installments INT,
    payment_value FLOAT,
    PRIMARY KEY (order_id, payment_sequential)
);
''')

cursor.execute('''
CREATE TABLE olist_order_reviews (
    review_id VARCHAR(32) PRIMARY KEY,
    order_id VARCHAR(32),
    review_score INT,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date DATETIME,
    review_answer_timestamp DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_orders (
    order_id VARCHAR(32) PRIMARY KEY,
    customer_id VARCHAR(32),
    order_status VARCHAR(50),
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATETIME,
    order_delivered_customer_date DATETIME,
    order_estimated_delivery_date DATETIME
);
''')

cursor.execute('''
CREATE TABLE olist_products (
    product_id VARCHAR(32) PRIMARY KEY,
    product_category_name VARCHAR(255),
    product_name_lenght FLOAT,
    product_description_lenght FLOAT,
    product_photos_qty FLOAT,
    product_weight_g FLOAT,
    product_length_cm FLOAT,
    product_height_cm FLOAT,
    product_width_cm FLOAT
);
''')

cursor.execute('''
CREATE TABLE olist_sellers (
    seller_id VARCHAR(32) PRIMARY KEY,
    seller_zip_code_prefix INT,
    seller_city VARCHAR(255),
    seller_state VARCHAR(2)
);
''')

cursor.execute('''
CREATE TABLE product_category_translation (
    product_category_name VARCHAR(255) PRIMARY KEY,
    product_category_name_english VARCHAR(255)
);
''')

#This saves the chnages to the databae.  Up unitl this point the executed SQL statement isn't stored, changes are not immediatley saved.
conn.commit()

print("Database and tables created successfully!")


Database and tables created successfully!


**STEP 2: Check Tables Created:**

Run the command to show the database tables created and the structure.

In [11]:

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)




Table: olist_customers
  Column: customer_id, Type: VARCHAR(32), NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: customer_unique_id, Type: VARCHAR(32), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_zip_code_prefix, Type: INT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_city, Type: VARCHAR(255), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: customer_state, Type: VARCHAR(2), NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: olist_geolocation
  Column: geolocation_zip_code_prefix, Type: INT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_lat, Type: FLOAT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_lng, Type: FLOAT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_city, Type: VARCHAR(255), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: geolocation_state, Type: VARCHAR(2), NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: olist_order

**STEP 3: Upload Files:**

Run this box multiple times to upload the relevant csv files. Or drag the files across to the Files window from your desktop.

In [ ]:


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


**STEP 4: Load CSV files into the database tables:**

This will populate the database tables with the data from the csv files.  No need to write INSERT statements.

You need to make sure the correct files are loaded into the corresponding tables.

In [3]:
import csv

def import_csv_to_table(csv_file, table_name):
    #opens the file aas read only 'r', doesn't allow the origianl csv to be changed.
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
            #? creates a placeholder for each column in the CSV file. ['?','?','?'] - Join makes it a string so it can then be inserted.
            # use of the '?' reduce risk of SQL injection
            placeholders = ', '.join(['?' for _ in row])
            #Assumes that the CSV and table have the same structure (this could be an issue) Would have to specify column names if different.
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Import data from CSV files into the relevant table - Student_Table goes into student table.  teh import_csv_to_table is the function, passing the two values across.
try:
    import_csv_to_table('olist_customers_dataset.csv', 'olist_customers')
    import_csv_to_table('olist_geolocation_dataset.csv', 'olist_geolocation')
    import_csv_to_table('olist_order_items_dataset.csv', 'olist_order_items')
    import_csv_to_table('olist_order_payments_dataset.csv', 'olist_order_payments')
    import_csv_to_table('olist_orders_dataset.csv', 'olist_orders')
    import_csv_to_table('olist_products_dataset.csv', 'olist_products')
    import_csv_to_table('olist_sellers_dataset.csv', 'olist_sellers')
    import_csv_to_table('product_category_name_translation.csv', 'product_category_translation')
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback changes if an error occurred



Data imported successfully!


**STEP 5: Check Data has loaded**

Query each database table and load the data into a dataframe and display the first 5 lines

In [4]:
import pandas as pd
# Query all three tables and load into pandas DataFrames
customers_df = pd.read_sql_query("SELECT * FROM olist_customers", conn)
location_df = pd.read_sql_query("SELECT * FROM olist_geolocation", conn)
orderItems_df = pd.read_sql_query("SELECT * FROM olist_order_items", conn)

#add in the other tables


# Show the first 5 lines of each DataFrame
print("Customers Table:")
print(customers_df.head(5))
print("\nLocations Table:")
print(location_df.head(5))
print("\nOrder Itmes Table:")
print(orderItems_df.head(5))




Customers Table:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  

Locations Table:
   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1037       -23.545621   

**ONLY RUN IF YOU NEED TO DELETE THE DATA IN THE TABLES**

If you run go back to **STEP 4** and re-run from there.

In [ ]:
# only run if you need to reset the tables without deleting the databae and starting again - then re-run the box previous box.
# Delete all data from the tables
cursor.execute("PRAGMA foreign_keys = OFF")
cursor.execute("DELETE FROM olist_customers")
cursor.execute("DELETE FROM olist_geolocation")
cursor.execute("DELETE FROM olist_order_items")
cursor.execute("DELETE FROM olist_order_payments")
cursor.execute("DELETE FROM olist_order_reviews")
cursor.execute("DELETE FROM olist_orders")
cursor.execute("DELETE FROM olist_products")
cursor.execute("DELETE FROM olist_sellers")
cursor.execute("DELETE FROM product_category_translation")
cursor.execute("PRAGMA foreign_keys = ON")

# Commit the changes
conn.commit()

conn.commit()
print("Database Deleted - restart.")



**STEP 6: SQL Select statements**

Run the following statements.  Please ask yoursefl the impact of each one before running.




In [5]:
import pandas as pd
#analyse the customer list. Think about how many customer, do they have duplicate names, take a look at the  data and bring some insight.

customers_df = pd.read_sql_query("SELECT COUNT(*) AS 'Number of customers' FROM olist_customers", conn)
customersCity_df = pd.read_sql_query("SELECT DISTINCT customer_city, COUNT(*) AS 'Number of customers per City' FROM olist_customers GROUP BY customer_city HAVING COUNT(*) >=100 ORDER BY COUNT(*) DESC", conn)

#What should the output be.
print(customers_df)
print(customersCity_df)

customersCity_df.to_csv('customersCity_df.csv', index=False)


   Number of customers
0                99441
          customer_city  Number of customers per City
0             sao paulo                         15540
1        rio de janeiro                          6882
2        belo horizonte                          2773
3              brasilia                          2131
4              curitiba                          1521
..                  ...                           ...
138            botucatu                           101
139         santa luzia                           100
140      patos de minas                           100
141    francisco morato                           100
142  balneario camboriu                           100

[143 rows x 2 columns]


In [8]:
#show number and value of orders for each customer

customersOrdersValue_df = pd.read_sql_query("""
SELECT olist_customers.customer_id,
       COUNT(DISTINCT olist_orders.order_id) AS total_orders,
       SUM(olist_order_items.price) AS total_order_value
FROM olist_customers, olist_orders, olist_order_items
WHERE olist_customers.customer_id = olist_orders.customer_id
AND olist_orders.order_id = olist_order_items.order_id
GROUP BY olist_customers.customer_id ORDER BY SUM(olist_order_items.price) DESC
""", conn)



print(customersOrdersValue_df)
customersOrdersValue_df.to_csv('customerOrdersValue_df.csv', index=False)


                            customer_id  total_orders  total_order_value
0      1617b1357756262bfa56ab541c47bc16             1           13440.00
1      ec5b2ba62e574342386871631fafd3fc             1            7160.00
2      c6e2731c5b391845f6800c97401a43a9             1            6735.00
3      f48d464a0baaea338cb25f816991ab1f             1            6729.00
4      3fd6777bbce08a352fddd04e4a7cc8f6             1            6499.00
...                                 ...           ...                ...
98661  d2c63ad286e3ca9dd69218008d61ff81             1               2.90
98662  184e8e8e48937145eb96c721ef1f0747             1               2.29
98663  a790343ca6f3fee08112d678b43aa7c5             1               2.20
98664  9f9d249355f63c5c1216a82b802452c1             1               0.85
98665  161b6d415e8b3413c6609c70cf405b5a             1               0.85

[98666 rows x 3 columns]


In [9]:
#where do the customers come from - location

cLocation_df = pd.read_sql_query("""
SELECT DISTINCT geolocation_city, COUNT(*) AS 'Customer City' FROM olist_geolocation GROUP BY geolocation_city ORDER BY COUNT(*) DESC
""", conn)
print(cLocation_df)
cLocation_df.to_csv('cLocation_df.csv', index=False)

cLocationCust_df = pd.read_sql_query("""
SELECT DISTINCT customer_city, COUNT(*) AS 'Customer City' FROM olist_customers GROUP BY customer_city ORDER BY COUNT(*) DESC
""", conn)
print(cLocationCust_df)

        geolocation_city  Customer City
0              sao paulo         135800
1         rio de janeiro          62151
2         belo horizonte          27805
3              são paulo          24918
4               curitiba          16593
...                  ...            ...
8006         abreulândia              1
8007              abatiá              1
8008      4o. centenario              1
8009  ...arraial do cabo              1
8010            * cidade              1

[8011 rows x 2 columns]
       customer_city  Customer City
0          sao paulo          15540
1     rio de janeiro           6882
2     belo horizonte           2773
3           brasilia           2131
4           curitiba           1521
...              ...            ...
4114         acucena              1
4115           acari              1
4116       acajutiba              1
4117   abdon batista              1
4118       abadiania              1

[4119 rows x 2 columns]


In [13]:
#look at which sellers are selling the most items

sellersSales_df = pd.read_sql_query("""
SELECT olist_sellers.seller_id AS 'Seller ID',
       COUNT(DISTINCT olist_orders.order_id) AS 'Total Orders',
       SUM(olist_order_items.price) AS 'Total Sales Value'
FROM olist_order_items, olist_orders, olist_sellers
WHERE olist_order_items.seller_id = olist_sellers.seller_id
AND olist_order_items.order_id = olist_orders.order_id
GROUP BY olist_sellers.seller_id
ORDER BY SUM(olist_order_items.price) DESC
""", conn)
print(sellersSales_df)
sellersSales_df.to_csv('sellersSales_df.csv', index=False)

#AVG order value for the seller
cSellersAVG_df = pd.read_sql_query("""
SELECT olist_sellers.seller_id AS 'Seller ID',
       COUNT(DISTINCT olist_orders.order_id) AS 'Total Orders',
       SUM(olist_order_items.price) AS 'Total Sales Value',
       AVG(olist_order_items.price) AS 'Average Sales Value'
FROM olist_order_items, olist_orders, olist_sellers
WHERE olist_order_items.seller_id = olist_sellers.seller_id
AND olist_order_items.order_id = olist_orders.order_id
GROUP BY olist_sellers.seller_id
ORDER BY AVG('olist_order_items.price') DESC
""", conn)

print(cSellersAVG_df)
cSellersAVG_df.to_csv('cSellersAVG_df.csv', index=False)

                             Seller ID  Total Orders  Total Sales Value
0     4869f7a5dfa277a7dca6462dcf3b52b2          1132          229472.63
1     53243585a1d6dc2643021fd1853d8905           358          222776.05
2     4a3ca9315b744ce9f8e9374361493884          1806          200472.92
3     fa1c13f2614d7b5c4749cbc52fecda94           585          194042.03
4     7c67e1448b00f6e969d365cea6b010ab           982          187923.89
...                                ...           ...                ...
3090  34aefe746cd81b7f3b23253ea28bef39             1               8.00
3091  702835e4b785b67a084280efca355756             1               7.60
3092  1fa2d3def6adfa70e58c276bb64fe5bb             1               6.90
3093  77128dec4bec4878c37ab7d6169d6f26             1               6.50
3094  cf6f6bc4df3999b9c6440f124fb2f687             1               3.50

[3095 rows x 3 columns]
                             Seller ID  Total Orders  Total Sales Value  \
0     ffff564a4f9085cd26170f47323937

In [14]:
#take a customer that has the most orders and pull out all the individual order items - is there any consistency - are they buying form the same categories every time.
customersOrders_df = pd.read_sql_query("""
SELECT olist_customers.customer_id,
       COUNT(DISTINCT olist_orders.order_id) AS total_orders,
       COUNT(olist_order_items.product_id) AS total_items_ordered,
       SUM(olist_order_items.price) AS total_order_value
FROM olist_customers, olist_orders, olist_order_items
WHERE olist_customers.customer_id = olist_orders.customer_id
AND olist_orders.order_id = olist_order_items.order_id
GROUP BY olist_customers.customer_id ORDER BY SUM(olist_order_items.price) DESC
""", conn)

print(customersOrders_df.head(5))

#get customer items fomr order
customerItems_df = pd.read_sql_query("""
SELECT olist_order_items.order_id AS 'Order ID',
       olist_order_items.product_id AS 'Product ID',
       olist_order_items.price AS 'Price',
       COUNT(*) OVER() AS 'Items Ordered',
       olist_products.product_category_name AS 'Product Category'
FROM olist_order_items, olist_orders, olist_customers, olist_products
WHERE olist_customers.customer_id = '1617b1357756262bfa56ab541c47bc16'
AND olist_customers.customer_id = olist_orders.customer_id
AND olist_orders.order_id = olist_order_items.order_id
AND olist_order_items.product_id = olist_products.product_id
ORDER BY olist_order_items.price DESC
""", conn)
print(customerItems_df)

                        customer_id  total_orders  total_items_ordered  \
0  1617b1357756262bfa56ab541c47bc16             1                    8   
1  ec5b2ba62e574342386871631fafd3fc             1                    4   
2  c6e2731c5b391845f6800c97401a43a9             1                    1   
3  f48d464a0baaea338cb25f816991ab1f             1                    1   
4  3fd6777bbce08a352fddd04e4a7cc8f6             1                    1   

   total_order_value  
0            13440.0  
1             7160.0  
2             6735.0  
3             6729.0  
4             6499.0  
                           Order ID                        Product ID   Price  \
0  03caa2c082116e1d31e67e9ae3700499  5769ef0a239114ac3a854af00df129e4  1680.0   
1  03caa2c082116e1d31e67e9ae3700499  5769ef0a239114ac3a854af00df129e4  1680.0   
2  03caa2c082116e1d31e67e9ae3700499  5769ef0a239114ac3a854af00df129e4  1680.0   
3  03caa2c082116e1d31e67e9ae3700499  5769ef0a239114ac3a854af00df129e4  1680.0   
4  03caa2c08

In [15]:
#How are customers paying - which payment methods are used the most

paymentMethods_df = pd.read_sql_query("""
SELECT olist_order_payments.payment_type AS 'Payment Method',
       COUNT(*) AS 'Number of Payments',
       ROUND(SUM(olist_order_payments.payment_value),2) AS 'Total Spent'
FROM olist_order_payments
GROUP BY olist_order_payments.payment_type
ORDER BY COUNT(*) DESC, ROUND(SUM(olist_order_payments.payment_value),2) DESC
""", conn)
print(paymentMethods_df)
paymentMethods_df.to_csv('paymentMethods_df.csv', index=False)

  Payment Method  Number of Payments  Total Spent
0    credit_card               76795  12542084.19
1         boleto               19784   2869361.27
2        voucher                5775    379436.87
3     debit_card                1529    217989.79
4    not_defined                   3         0.00


In [16]:
#if possible, look at the payment methods and location - Identify the area that spends the highest amount - you could look at hteh product categories as well.

paymentByLocation_df = pd.read_sql_query("""
SELECT olist_customers.customer_city AS 'City',
       olist_order_payments.payment_type AS 'Payment Method',
       COUNT(*) AS 'Number of Payments',
       ROUND(SUM(olist_order_payments.payment_value),2) AS 'Total Spent'
FROM olist_order_payments, olist_orders, olist_customers
WHERE olist_order_payments.order_id = olist_orders.order_id
AND olist_orders.customer_id = olist_customers.customer_id
GROUP BY olist_order_payments.payment_type, olist_customers.customer_city
ORDER BY SUM(olist_order_payments.payment_value) DESC
""", conn)

# Print formatted output
print(paymentByLocation_df)
paymentByLocation_df.to_csv('paymentByLocation_df.csv', index=False)


                         City Payment Method  Number of Payments  Total Spent
0                   sao paulo    credit_card               12226   1757931.79
1              rio de janeiro    credit_card                5525    942028.03
2                   sao paulo         boleto                2783    358042.07
3              belo horizonte    credit_card                2233    343387.39
4                    brasilia    credit_card                1693    291241.56
...                       ...            ...                 ...          ...
7494                   iguape        voucher                   1         1.65
7495          santo anastacio        voucher                   1         1.29
7496          antonio pereira        voucher                   1         0.92
7497  sao sebastiao de campos    credit_card                   1         0.19
7498                sao paulo    not_defined                   3         0.00

[7499 rows x 4 columns]


In [18]:
#Range of payment by location
paymentByLocationRange_df = pd.read_sql_query("""
SELECT olist_customers.customer_city AS 'City',
       olist_customers.customer_state AS 'State',
       olist_order_payments.payment_type AS 'Payment Method',
       COUNT(*) AS 'Number of Payments',
       SUM(olist_order_payments.payment_value) AS 'Total Spent',
       MAX(olist_order_payments.payment_value) AS 'Max Payment',
       MIN(olist_order_payments.payment_value) AS 'Min Payment',
       (MAX(olist_order_payments.payment_value) - MIN(olist_order_payments.payment_value)) AS 'Payment Range'
FROM olist_order_payments, olist_orders, olist_customers
WHERE olist_order_payments.order_id = olist_orders.order_id
AND olist_orders.customer_id = olist_customers.customer_id
GROUP BY olist_customers.customer_city, olist_customers.customer_state, olist_order_payments.payment_type
ORDER BY olist_customers.customer_state ASC, SUM(olist_order_payments.payment_value) DESC
""", conn)

# Round monetary values for better readability
paymentByLocationRange_df[['Total Spent', 'Max Payment', 'Min Payment', 'Payment Range']] = paymentByLocationRange_df[['Total Spent', 'Max Payment', 'Min Payment', 'Payment Range']].round(2)

# Print formatted output
paymentByLocationRange_df.to_csv('paymentByLocationRange_df.csv',index=False)
print(paymentByLocationRange_df)


                  City State Payment Method  Number of Payments  Total Spent  \
0           rio branco    AC    credit_card                  55     13349.94   
1           rio branco    AC         boleto                  14      2871.68   
2      cruzeiro do sul    AC         boleto                   2      1036.78   
3           rio branco    AC        voucher                   4       706.85   
4     senador guiomard    AC    credit_card                   1       446.18   
...                ...   ...            ...                 ...          ...   
7718         combinado    TO    credit_card                   1        54.63   
7719              pium    TO    credit_card                   1        45.63   
7720    tocantinopolis    TO         boleto                   1        38.12   
7721        araguatins    TO        voucher                   1        33.77   
7722       barrolandia    TO    credit_card                   1        28.77   

      Max Payment  Min Payment  Payment

In [19]:
#What's the average order spend in the categories - show the category name in English

avgOrderSpendByCategory_df = pd.read_sql_query("""
SELECT olist_products.product_category_name AS 'Product Category',
       COUNT(DISTINCT olist_orders.order_id) AS 'Number of Orders',
       SUM(olist_order_items.price) AS 'Total Sales Value',
       ROUND(SUM(olist_order_items.price) / COUNT(DISTINCT olist_orders.order_id), 2) AS 'Average Order Spend'
FROM olist_order_items, olist_orders, olist_products
WHERE olist_order_items.order_id = olist_orders.order_id
AND olist_order_items.product_id = olist_products.product_id
GROUP BY olist_products.product_category_name
ORDER BY ROUND(SUM(olist_order_items.price) / COUNT(DISTINCT olist_orders.order_id), 2) DESC
""", conn)

# Print formatted output
print(avgOrderSpendByCategory_df.to_string(index=False))
avgOrderSpendByCategory_df.to_csv('avgOrderSpendByCategory_df.csv', index=False)


                              Product Category  Number of Orders  Total Sales Value  Average Order Spend
                                           pcs               181          222963.13              1231.84
                   portateis_casa_forno_e_cafe                75           47445.71               632.61
                            eletrodomesticos_2               234          113317.74               484.26
                     agro_industria_e_comercio               182           72530.47               398.52
                         instrumentos_musicais               628          191498.88               304.93
                               eletroportateis               630          190648.58               302.62
 portateis_cozinha_e_preparadores_de_alimentos                14            3968.53               283.47
                                telefonia_fixa               217           59583.00               274.58
              construcao_ferramentas_seguranca         

In [20]:
#get categories in English

avgOrderSpendByCategory_df = pd.read_sql_query("""
SELECT pct.product_category_name_english AS 'Product Category',
       COUNT(DISTINCT olist_orders.order_id) AS 'Number of Orders',
       SUM(olist_order_items.price) AS 'Total Sales Value',
       ROUND(SUM(olist_order_items.price) / COUNT(DISTINCT olist_orders.order_id), 2) AS 'Average Order Spend'
FROM olist_order_items, olist_orders, olist_products, product_category_translation pct
WHERE olist_order_items.order_id = olist_orders.order_id
AND olist_order_items.product_id = olist_products.product_id
AND olist_products.product_category_name = pct.product_category_name
GROUP BY pct.product_category_name_english
ORDER BY ROUND(SUM(olist_order_items.price) / COUNT(DISTINCT olist_orders.order_id), 2) DESC
""", conn)

# Print formatted output
print(avgOrderSpendByCategory_df.to_string(index=False))


                       Product Category  Number of Orders  Total Sales Value  Average Order Spend
                              computers               181          222963.13              1231.84
  small_appliances_home_oven_and_coffee                75           47445.71               632.61
                      home_appliances_2               234          113317.74               484.26
             agro_industry_and_commerce               182           72530.47               398.52
                    musical_instruments               628          191498.88               304.93
                       small_appliances               630          190648.58               302.62
                        fixed_telephony               217           59583.00               274.58
              construction_tools_safety               167           40544.52               242.78
                       air_conditioning               253           55024.96               217.49
                    